In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms, datasets, models
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Define the path to your dataset
path = '/home/rishabh.mondal/R_P_project/Faulty_solar_panel'

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=path, transform=transform)

# Define the number of splits for cross-validation
n_splits = 4  # 4-fold cross-validation

# Initialize stratified k-fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [2]:


# Lists to store metrics for plotting
train_losses = []
val_losses = []
accuracies = []

# Training loop for each fold
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(dataset.samples, dataset.targets)):
    print(f"Training Fold {fold + 1}/{n_splits}")

    # Create data loaders for the current fold
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=264, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=264, sampler=val_sampler)

    print(len(train_loader), len(val_loader))
    # # Define the EfficientNet model
    model = models.efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(1280, len(dataset.classes))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0

        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training'):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = total_correct / total_samples
        train_losses.append(train_loss)

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            all_predicted = []
            all_labels = []

            for images, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation'):
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            val_losses.append(val_loss / len(val_loader))
            accuracy = correct / total
            accuracies.append(accuracy)

            # Calculate precision, recall, and F1 score
            precision = precision_score(all_labels, all_predicted, average='weighted')
            recall = recall_score(all_labels, all_predicted, average='weighted')
            f1 = f1_score(all_labels, all_predicted, average='weighted')
            if epoch % 25 == 0:
                print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, '
                    f'Val Loss: {val_loss / len(val_loader)}, Val Accuracy: {accuracy}, '
                    f'Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

# Plotting loss and accuracy curves
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


Training Fold 1/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50 - Training:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/50 - Validation:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/50, Train Loss: 1.5305240948994954, Train Accuracy: 0.41478129713423834, Val Loss: 1.3740423917770386, Val Accuracy: 0.49099099099099097, Precision: 0.5642747436640566, Recall: 0.49099099099099097, F1 Score: 0.45311589955147547


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/50 - Training:   0%|          | 0/3 [00:00<?, ?it/s]